In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.multiclass import OneVsRestClassifier
from bs4 import BeautifulSoup
import re
stop_words = stopwords.words('english')

/home/thakkark1313/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
train = pd.read_csv('../input/trainpro.csv')
test = pd.read_csv('../input/testpro.csv')
sample = pd.read_csv('../input/sample_submission.csv')

In [3]:
def clean_comment(row):        
    input=row['comment_text']
    cleaned_html=BeautifulSoup(input).get_text()
    cleaned_special_chr=re.sub('[^a-zA-Z]',' ',cleaned_html)
    cleaned_special_chr = re.sub(r'[?|$|.|!]',r'',cleaned_special_chr)
    lower=cleaned_special_chr.lower().split()
    clean_stop_word=[each_char for each_char  in lower if each_char not in set(stopwords.words("english"))]
    return ' '.join(clean_stop_word)
print("pre train")
train['comment_text'] = train.apply(clean_comment,axis = 1)
print("pre test")
test['comment_text'] = test.apply(clean_comment,axis=1)

pre train


/home/thakkark1313/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/thakkark1313/miniconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/home/thakkark1313/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://en.wikipedia.org/wiki/Wikipedia_talk:No_original_research/archive15#YouTube_art_as_primary_source" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL,

pre test


/home/thakkark1313/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://en.wikipedia.org/wiki/Half-Life_%28series%29" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/thakkark1313/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.google.ca/search?hl=en&source;=hp&q;=%22book+identifier%22+edu.ca&btnG;=Google+Search&meta;=cr%3DcountryCA&aq;=f&oq;=" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/thakkark1313/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://es.wikipedia.org/wiki/Usuario_Discusi%C3%B3n:Sa

In [3]:
train.fillna("Unknown",inplace = True)
test.fillna("Unknown", inplace = True)
xtrain = train.comment_text.values
xtest = test.comment_text.values

In [4]:
col_list = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y = train[col_list]

In [20]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xtest))
print("fitting training data")
xtrain_tfv =  tfv.transform(xtrain) 
print("fitting testing data")
xtest_tfv = tfv.transform(xtest)

fitting training data
fitting testing data


In [21]:
predictions = OneVsRestClassifier(LogisticRegression(C=1.0)).fit(xtrain_tfv, y).predict_proba(xtest_tfv)

In [ ]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xtest))
xtrain_ctv =  ctv.transform(xtrain) 
xtest_ctv = ctv.transform(xtest)

In [ ]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, y)
predictions1 = clf.predict_proba(xtest_ctv)

In [5]:
embeddings_index = {}
f = open('../input/glove.840B.300d.txt',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

2196017it [02:53, 12635.85it/s]


In [6]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [7]:
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xtest_glove = [sent2vec(x) for x in tqdm(xtest)]

100%|██████████| 153164/153164 [01:05<00:00, 2355.27it/s]


In [8]:
xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)
print(xtrain_glove.shape)
print(xtest_glove.shape)
print(y.shape)

(159571, 300)
(153164, 300)
(159571, 6)


In [9]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xtest_glove_scl = scl.transform(xtest_glove)

In [36]:
ytrain_enc = np_utils.to_categorical(y)
print(ytrain_enc.shape)
print(y.shape)
print(ytrain_enc.shape)

(957426, 2)
(159571, 6)
(957426, 2)


In [11]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xtest))
xtrain_seq = token.texts_to_sequences(xtrain)
xtest_seq = token.texts_to_sequences(xtest)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [12]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 286061/286061 [00:00<00:00, 430333.26it/s]


In [23]:
'''print(xtrain_pad.shape)
print(y.shape)
print(len(word_index))
print(ytrain_enc.shape)'''
print(type(xtrain_pad))
ytrain = np.array(y)

<class 'numpy.ndarray'>


In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

model.add(Bidirectional(GRU(100, dropout=0.3, recurrent_dropout=0.3,return_sequences = True)))
model.add(Bidirectional(GRU(100, dropout=0.3, recurrent_dropout=0.3,return_sequences = True)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('sigmoid'))
          
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=y, batch_size=512, epochs=3,verbose=1)

In [14]:
model1 = Sequential()
model1.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model1.add(SpatialDropout1D(0.3))
model1.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))

model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.8))

model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.8))

model1.add(Dense(6))
model1.add(Activation('sigmoid'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
model1 = Sequential()
model1.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model1.add(SpatialDropout1D(0.3))
model1.add(GRU(100, dropout=0.3, recurrent_dropout=0.3,return_sequences=True))
model1.add(GRU(100, dropout=0.3, recurrent_dropout=0.3,return_sequences=True))


model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.8))

model1.add(Dense(6))
model1.add(Activation('relu'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [37]:
#earlystop = EarlyStopping(, min_delta=0, patience=3, verbose=0, mode='auto')
model1.fit(xtrain_pad, y=y, batch_size=512, epochs=2,verbose=1)

ValueError: Error when checking target: expected activation_3 to have 3 dimensions, but got array with shape (159571, 6)

In [48]:
from keras.layers import GlobalMaxPool1D
model1 = Sequential()
model1.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model1.add(SpatialDropout1D(0.3))
model1.add(GlobalMaxPool1D())
model1.add(BatchNormalization())
model1.add(GlobalMaxPool1D())
model1.add(BatchNormalization())
model1.add(Dense(1024))
model1.add(Dropout(0.5))
model1.add(Dense(6,activation='sigmoid'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

ValueError: Input 0 is incompatible with layer global_max_pooling1d_5: expected ndim=3, found ndim=2

In [ ]:
model1.fit(xtrain_pad, y=ytrain, batch_size=512, epochs=20,verbose=1)

In [46]:
#predictions = model.predict(xtest_pad)
predictions1 = model1.predict(xtest_pad)
sample1 = sample.copy()
#sample[col_list] = predictions
sample1[col_list] = predictions1
#sample.to_csv('b1.csv',index=False)
sample1.to_csv('cnn.csv',index=False)

In [ ]:
predictions = model.predict(xtest_pad)
sample[col_list] = predictions
sample.to_csv('log.csv',index=False)